In [0]:
 pip install keras_tqdm

Imports

In [0]:
import seaborn as sns
import pickle
from PIL import Image
from keras.preprocessing.image import img_to_array

from keras import applications
from keras import backend as K
from keras import optimizers
from keras.callbacks import ModelCheckpoint, History
from keras.models import Model
from keras.preprocessing.image import ImageDataGenerator

import matplotlib.pyplot as plt
from math import *
import copy

import numpy as np
from tensorflow.keras.preprocessing.image import ImageDataGenerator, NumpyArrayIterator, load_img
from tensorflow.keras.models import Model
from tensorflow.keras.applications.inception_v3 import preprocess_input
from autoencoder_keras import Autoencoder
import os
import cv2

os.environ['TF_CPP_MIN_LOG_LEVEL'] = '2' 


Exibir informações do plot

In [0]:
my_image = mpimg.imread( "/home/analimazn/Projetos/fatec/graduate/example-code/data/train/GOESR_RET_CH13_IRCOL0_20190529_1640.png")

my_image_copy_1= copy.deepcopy(my_image)
my_rotated_image= copy.deepcopy(my_image)
my_image_difference= copy.deepcopy(my_image)


print('Reading the number of rows and columns: ')
number_of_rows = len(my_image)
number_of_columns = len(my_image[0])
#Remark: % stands for string formatting operator
print('Number of rows: %s' %(number_of_rows), end='\n')
print('Number of columns: %s ' %(number_of_columns), end='\n')


plt.figure(0)
imgplot = plt.imshow(my_image)
plt.title('Original image')
plt.grid()

Exibir informações do Model

In [0]:
scores = Model.evaluate(x, y, verbose=1)
print(scores)

NumpyArrayIterator

In [0]:
test = NumpyArrayIterator(x, y, train_generator, save_to_dir='/home/analimazn/Projetos/fatec/graduate/example-code/images')

Gerar imagens menores

In [0]:
# train_datagen=ImageDataGenerator(
#    preprocessing_function=preprocess_input
# )

# test_datagen=ImageDataGenerator(
#     preprocessing_function=preprocess_input,
#     width_shift_range=0.2,
#     height_shift_range=0.2,
#     shear_range=0.2,
#     zoom_range=0.2
# )

# jf_datagen=ImageDataGenerator(
#     preprocessing_function=preprocess_input,
#     horizontal_flip = True
# )

# train_datagen = ImageDataGenerator(
#           rescale=1./255,
#           shear_range=0.2,
#           zoom_range=0.2,
#           horizontal_flip=True)

# test_datagen = ImageDataGenerator(preprocessing_function=preprocess_input)

# train_generator=train_datagen.flow_from_directory('/home/analimazn/Projetos/fatec/graduate/example-code/data', 
#                                                class_mode='input', 
#                                                save_to_dir='/home/analimazn/Projetos/fatec/graduate/example-code/images',
#                                                color_mode='rgba')

# train_generator = train_datagen.flow_from_directory(
#     '/home/analimazn/Projetos/fatec/graduate/example-code/data',
#     target_size=(150, 150),
#     batch_size=32,
#     class_mode='input',
#     color_mode='rgba')


# validation_generator = test_datagen.flow_from_directory(
#     '/home/analimazn/Projetos/fatec/graduate/example-code/data',
#     class_mode='input',
#     color_mode='rgba')


img_width, img_height = 100, 100
train_data_dir = "./data"
validation_data_dir = "./data"
batch_size = 64
nb_epoch = 10
nb_channels= 6

train_datagen = ImageDataGenerator(
        rescale=1./255,
        shear_range=0.2,
        zoom_range=0.2,
        horizontal_flip=True)

test_datagen = ImageDataGenerator(rescale=1./255)

train_generator = train_datagen.flow_from_directory(
        os.path.join(train_data_dir),
        target_size=(img_width, img_height),
        batch_size=batch_size,
        class_mode=None, shuffle=True)

validation_generator = test_datagen.flow_from_directory(
        os.path.join(validation_data_dir),
        target_size=(img_width, img_height),
        batch_size=batch_size,
        class_mode=None, shuffle=True)

Exemplo de subplot

In [0]:
n = 2
plt.figure(figsize=(60, 4))
for i in range(n):
    # display original
    ax = plt.subplot(2, n, i+1)
    plt.title('Train image')
    plt.imshow(x_test[i])
    plt.gray()
    #ax.get_xaxis().set_visible(False)
    #ax.get_yaxis().set_visible(False)

plt.figure(figsize=(30, 4))
for i in range(n):
    # display reconstruction
    ax = plt.subplot(2, n, i + n+1)
    plt.title('Reconstruction image')
    plt.imshow(decoded_imgs.reshape(1, 60000))  
    plt.gray()
    #ax.get_xaxis().set_visible(False)
    #ax.get_yaxis().set_visible(False)
plt.show()


Exemplo de modelo salvo

In [0]:
checkpoint = ModelCheckpoint(filepath = "model_weights_ae_fc.h5", save_best_only=True,monitor="val_loss", mode="min" )
history = History()


Exemplo de métricas impressas

In [0]:
plt.figure(figsize=(15,10))
plt.plot(history.epoch,history.history["val_loss"])
plt.plot(history.epoch,history.history["loss"])
plt.title("Validation loss and loss per epoch",fontsize=18)
plt.xlabel("epoch",fontsize=18)
plt.ylabel("loss",fontsize=18)
plt.legend(['Validation Loss','Training Loss'],fontsize=14)
plt.show()

Exemplo de for usando subdiretórios

In [0]:
DATADIR = "/home/analimazn/Projetos/fatec/graduate/example-code/data"
CATEGORIES = ['color', 'gray']

for category in CATEGORIES:
    path = os.path.join(DATADIR, category)
    class_num = CATEGORIES.index(category)
    for img in os.listdir(path):
        try:
            my_image = mpimg.imread(os.path.join(path, img))
            x_train = my_image.astype('float64') / 255
            x_test = my_image.astype('float64') / 255
            x_train = x_train.reshape((len(x_train), np.prod(x_train.shape[1:])))
            x_test = x_test.reshape((len(x_test), np.prod(x_test.shape[1:])))
            
            #     print(x_train)
            #     print(x_train)
            
            autoencoder = Autoencoder(x_train.shape[1], 64)
            autoencoder.train(x_train, x_test, 256, 10)
            encoded_imgs = autoencoder.getEncodedImage(x_train)
            decoded_imgs = autoencoder.getDecodedImage(encoded_imgs)
            
            plt.figure()
            imgplot = plt.imshow(x_train)
            plt.title('Train image')
            plt.grid()
            
            plt.figure()
            imgplot = plt.imshow(x_test)
            plt.title('Test image')
            plt.grid()
    

        except Exception as e:
            pass
            

Exemplo para criar um array de treino

In [0]:
DATADIR = "/home/analimazn/Projetos/fatec/graduate/example-code/data"
CATEGORIES = ['color', 'gray']

training_data = []

def create_training_data():
    for category in CATEGORIES:
        path = os.path.join(DATADIR, category)
        class_num = CATEGORIES.index(category)
        for img in os.listdir(path):
            try:
                img_array = mpimg.imread(os.path.join(path, img))
                print(img_array)
                training_data.append([img_array, class_num])
            except Exception as e:
                pass
   
    return training_data
    

Classe do DeepAutoencoder

In [0]:
class DeepAutoencoder(object):
    
    def __init__(self, input_dim, encoded_dim):
        self.input_img = Input(shape=(input_dim,))
        self.autoencoder = Sequential()

        # Encoder Layers
        self.autoencoder.add(Dense(8 * encoded_dim, input_shape=(input_dim,), activation='relu'))
        self.autoencoder.add(Dense(6 * encoded_dim, activation='relu'))
        self.autoencoder.add(Dense(4 * encoded_dim, activation='relu'))
        self.autoencoder.add(Dense(2 * encoded_dim, activation='relu'))
        self.autoencoder.add(Dense(encoded_dim, activation='relu'))

        self.autoencoder.add(Dense(2 * encoded_dim, activation='relu'))
        self.autoencoder.add(Dense(4 * encoded_dim, activation='relu'))
        self.autoencoder.add(Dense(6 * encoded_dim, activation='relu'))
        self.autoencoder.add(Dense(8 * encoded_dim, activation='relu'))
        self.autoencoder.add(Dense(input_dim, activation='sigmoid'))
    
        # Decoder Layers
        encoder_layer1 = self.autoencoder.layers[0]
        encoder_layer2 = self.autoencoder.layers[1]
        encoder_layer3 = self.autoencoder.layers[2]
        self.encoder = Model(self.input_img, encoder_layer3(encoder_layer2(encoder_layer1(self.input_img))))
        self.autoencoder.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])

        self.autoencoder.summary()
        self.encoder.summary()
    
    def train(self, input_train, input_test, batch_size, epochs):    
        self.autoencoder.fit(input_train, input_train,
                            epochs=epochs,
                            batch_size=batch_size,
                            shuffle=True,
                            validation_data=(input_test, input_test))
    
    def getEncodedImage(self, image):
        encoded_img = self.encoder.predict(image)
        return encoded_img
    
    def getDecodedImage(self, encoded_img):
        decoded_img = self.autoencoder.predict(encoded_img)
        return decoded_img


Main usando a classe DeepAutoencoder

In [0]:
def main_class():
    images = []
    for img in os.listdir(DIR_RESIZED_IMAGES):
        try: 
            # Normalização das imagens     
            my_image_original = mpimg.imread((os.path.join(DIR_RESIZED_IMAGES, img)))
            max_original_value = float(my_image_original.max())
            x_original = my_image_original.astype('float32') / max_original_value
            x_original = x_original.reshape((len(x_original), np.prod(x_original.shape[1:])))

            print(x_original.shape[1])
            
            autoencoder = DeepAutoencoder(x_original.shape[1], 64)
            autoencoder.train(x_original, x_original, 128, 5)
                        
            encoded_imgs = autoencoder.getEncodedImage(x_original)
            decoded_imgs = autoencoder.getDecodedImage(encoded_imgs)
            
            # Dimensões da imagem original
            to_reshape = my_image_original.shape

            # Redimensionamento das imagens obtidas
            x_original_result = x_original.reshape(to_reshape[0], to_reshape[1], to_reshape[2])
            decoded_imgs_result = decoded_imgs.reshape(to_reshape[0], to_reshape[1], to_reshape[2])      

            images.append(x_original_result)
            images.append(encoded_imgs)
            images.append(decoded_imgs_result)

            mpimg.imsave(DIR_RESULTS + 'original_' + img, x_original_result)
            mpimg.imsave(DIR_RESULTS + 'encoded_imgs_' + img, encoded_imgs)
            mpimg.imsave(DIR_RESULTS + 'decoded_imgs_' + img, decoded_imgs_result)

        except Exception as e:
            print(e)
            pass
        
    return images